In [1]:
import os, sys

import cv2
import numpy as np

import torch
from torchsummary import summary

from nets.mobilenet_v2 import MobileNetV2

In [2]:
def load_model(num_classes, model_path):
    model = MobileNetV2(n_class=num_classes, width_mult=1.0, test=True)
    model.load_state_dict(torch.load(model_path))
    return model

In [3]:
def preprocess_data(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    input_tensor = img - 127.5
    input_tensor = input_tensor / 127.5
    input_tensor = np.transpose(input_tensor, (2, 0, 1))
    input_tensor = np.expand_dims(input_tensor, 0)
    input_tensor = torch.from_numpy(input_tensor).float()
    return input_tensor

In [4]:
def predict(model, input_tensor):
    model.eval()
    with torch.no_grad():
        model.cuda()
        input_tensor = input_tensor.cuda()
        output = model(input_tensor)
        _, pred_label = torch.max(output, 1)
    return pred_label.detach().cpu().numpy()

In [5]:
testdata_path = './data/female_hair/val'
# model_path = './pretrained_weights/female_hair_mobilenetv2_epoch_200.pth' 
model_path='./results/check_points/female_hair_mobilenetv2_epoch_98.pth'
num_classes = len([_ for _ in os.listdir(testdata_path) if os.path.isdir(os.path.join(testdata_path, _))])

In [6]:
model = load_model(num_classes, model_path)
summary(model.cuda(), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
             ReLU6-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]             288
       BatchNorm2d-5         [-1, 32, 112, 112]              64
             ReLU6-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 16, 112, 112]             512
       BatchNorm2d-8         [-1, 16, 112, 112]              32
  InvertedResidual-9         [-1, 16, 112, 112]               0
           Conv2d-10         [-1, 96, 112, 112]           1,536
      BatchNorm2d-11         [-1, 96, 112, 112]             192
            ReLU6-12         [-1, 96, 112, 112]               0
           Conv2d-13           [-1, 96, 56, 56]             864
      BatchNorm2d-14           [-1, 96,

In [7]:
total_count = 0
total_sample = 0
for i in range(num_classes):
    hair_type = "{:02d}".format(i)
    print("Predict hair type:", hair_type)
    hair_path = os.path.join(testdata_path, hair_type)
    imgs = [f for f in os.listdir(hair_path) if f.endswith('.jpg')]
    count = 0
    sample = 0
    for img in imgs:
        img = cv2.imread(os.path.join(hair_path, img))
        input_tensor = preprocess_data(img)
        label = predict(model, input_tensor)
        sample += 1
        if label == i:
            count += 1
    total_count += count
    total_sample += sample
    print("Accuracy:", count / sample)
print("Total Accuray:", total_count / total_sample)

Predict hair type: 00
Accuracy: 0.7642276422764228
Predict hair type: 01
Accuracy: 0.48333333333333334
Predict hair type: 02
Accuracy: 0.7647058823529411
Predict hair type: 03
Accuracy: 0.7105263157894737
Predict hair type: 04
Accuracy: 0.6086956521739131
Predict hair type: 05
Accuracy: 0.4925373134328358
Predict hair type: 06
Accuracy: 0.5
Predict hair type: 07
Accuracy: 0.6285714285714286
Predict hair type: 08
Accuracy: 0.7121212121212122
Predict hair type: 09
Accuracy: 0.6
Predict hair type: 10
Accuracy: 0.5211267605633803
Predict hair type: 11
Accuracy: 0.8461538461538461
Predict hair type: 12
Accuracy: 0.5869565217391305
Predict hair type: 13
Accuracy: 0.5
Predict hair type: 14
Accuracy: 0.796875
Predict hair type: 15
Accuracy: 0.7023809523809523
Predict hair type: 16
Accuracy: 0.47540983606557374
Predict hair type: 17
Accuracy: 0.5846153846153846
Predict hair type: 18
Accuracy: 0.38571428571428573
Predict hair type: 19
Accuracy: 0.559322033898305
Predict hair type: 20
Accuracy: 0